# Aplicando CNN para classificação de fissura em concreto armado e pavimento

Nestes notebooks será aplicado redes pré-treinadas para detecção de fissura, neste dataset de imagens, o foco é comparar a redes pré-treinadas e ver qual seria a rede com o melhor desempenho

In [1]:
#importando as blibiotecas

from tensorflow import keras
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16, ResNet50

import tensorflow as tf

In [2]:
# importando a blibioteca split-folders para dividir as imagens em treino e teste
!pip install split-folders

import splitfolders
splitfolders.ratio('/kaggle/input/concrete-and-pavement-crack-images', 
                   output="output", seed=1337, ratio=(.7, 0.3))

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Copying files: 30000 files [00:31, 938.60 files/s] 


In [3]:
BATCH = 26
IMG_WIDTH = 96
IMG_HEIGHT = 96

In [4]:
train_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/train",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH,label_mode= 'categorical'
)

test_loader = tf.keras.preprocessing.image_dataset_from_directory(
"./output/val",
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=BATCH, label_mode= 'categorical')

Found 21000 files belonging to 2 classes.
Found 9000 files belonging to 2 classes.


## Criando o modelo ResNet50

In [5]:
model = ResNet50(input_shape=(96, 96, 3), weights='imagenet', include_top=False,
             pooling='max',classifier_activation='softmax')
model.layers[0].trainable = False
model = Model(inputs=model.input, outputs=model.get_layer(index=-2).output)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [6]:
#Colocando as duas últimas camadas
newTop = Sequential()
newTop.add(Flatten(input_shape=model.output_shape[1:]))
newTop.add(Dense(2, activation='softmax'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
metricas = ['accuracy',
            keras.metrics.Precision(name='Pre'),keras.metrics.Recall(name='Rec')] 
#métricas que vão ser utilizadas para teste

In [8]:
model = Model(inputs=model.input, outputs=newTop(model.output))
model.summary()

optimizer = 'adam'
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=metricas)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 96, 96, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 102, 102,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 48, 48,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 48, 48,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 48, 48,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 50, 50,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 24, 24,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 24, 24,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 24, 24,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 24, 24,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 24, 24,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 24, 24,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 24, 24,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 24, 24,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 24, 24,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 24, 24,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 24, 24,    │      1,024 │ conv2_block1_3_c

 Total params: 23,624,578 (90.12 MB)

 Trainable params: 23,571,458 (89.92 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [9]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_loader.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_loader.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
callback = keras.callbacks.EarlyStopping(monitor='loss',
                                              patience=2)
history = model.fit(train_dataset,epochs=5,batch_size=100,callbacks=[callback])

Epoch 1/5


I0000 00:00:1733234813.483591    1056 service.cc:145] XLA service 0x7b571c001850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733234813.483639    1056 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  2/808 ━━━━━━━━━━━━━━━━━━━━ 48s 60ms/step - Pre: 0.6346 - Rec: 0.6346 - accuracy: 0.6346 - loss: 0.7730    

I0000 00:00:1733234833.579853    1056 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_41', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_40', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_36', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_35', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_32', 4 bytes spill stores, 12 bytes spill loads

I0000 00:00:1733234833.622821    1056 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


624/808 ━━━━━━━━━━━━━━━━━━━━ 14s 78ms/step - Pre: 0.9466 - Rec: 0.9466 - accuracy: 0.9466 - loss: 0.2024

I0000 00:00:1733234881.920719    1056 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_41', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_40', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_36', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_35', 4 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_32', 4 bytes spill stores, 12 bytes spill loads



808/808 ━━━━━━━━━━━━━━━━━━━━ 113s 71ms/step - Pre: 0.9522 - Rec: 0.9522 - accuracy: 0.9522 - loss: 0.1789
Epoch 2/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 40s 49ms/step - Pre: 0.9900 - Rec: 0.9900 - accuracy: 0.9900 - loss: 0.0319
Epoch 3/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - Pre: 0.9940 - Rec: 0.9940 - accuracy: 0.9940 - loss: 0.0218
Epoch 4/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - Pre: 0.9921 - Rec: 0.9921 - accuracy: 0.9921 - loss: 0.0243
Epoch 5/5
808/808 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - Pre: 0.9644 - Rec: 0.9644 - accuracy: 0.9644 - loss: 0.0958


In [11]:
model.evaluate(test_dataset)

347/347 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - Pre: 0.9887 - Rec: 0.9887 - accuracy: 0.9887 - loss: 0.0373


[0.040332552045583725,
 0.9872221946716309,
 0.9872221946716309,
 0.9872221946716309]